# Preamble

In [8]:
import $file.sparksession
import sparksession._
import spark.implicits._
import org.apache.spark._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.types._, func._

import $file.$           

import sparksession._

import spark.implicits._

import org.apache.spark._

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.types._, func._

# On `DataFrame`s

We can create datasets from external data sources using different formats, e.g. Json, parquet, CSV, etc. 

In [9]:
val people: DataFrame = spark.read.json("data/people.json")

json at cmd8.sc:1

1 / 1

people: DataFrame = [age: bigint, name: string]

Note that we created a `DataFrame`, not a `Dataset`. Dataframes are like datasets, i.e. programs to generate distributed data sets, but *dynamically typed*. This means that, at compile time, Scala only knows that a dataframe consists of `Row`s.

In [10]:
people.collect

collect at cmd9.sc:1

1 / 1

res9: Array[Row] = Array(
  [2000,YiHui],
  [2000,Javier],
  [2003,Gabriel],
  [2004,Noelia]
)

In fact, a `DataFrame` is defined as an alias of `Dataset`: 

In [11]:
val peopleDs: Dataset[Row] = people

peopleDs: Dataset[Row] = [age: bigint, name: string]

But the type of the information to be processed is there! 

In [12]:
people.schema
people.printSchema

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



res11_0: StructType = StructType(
  StructField("age", LongType, true, {}),
  StructField("name", StringType, true, {})
)

and we can convert a dataframe into a dataset: 

In [13]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)

case class Person(name: String, age: Long)

val peopleDs: Dataset[Person] = people.as[Person]

defined class Person
peopleDs: Dataset[Person] = [age: bigint, name: string]

In [14]:
peopleDs.show
people.show

show at cmd13.sc:1

1 / 1

+----+-------+
| age|   name|
+----+-------+
|2000|  YiHui|
|2000| Javier|
|2003|Gabriel|
|2004| Noelia|
+----+-------+



show at cmd13.sc:2

1 / 1

+----+-------+
| age|   name|
+----+-------+
|2000|  YiHui|
|2000| Javier|
|2003|Gabriel|
|2004| Noelia|
+----+-------+



# Untyped transformations

The `Dataset` API includes a section on _untyped transformations_. These are transformations that are not defined over the Scala types but over the inner Spark SQL types (i.e. `StructType`s). More exactly, these could be named *dynamically typed transformations*.

These transformations are in close corresponde with their SQL counterparts: `SELECT`, `WHERE`, `GROUP BY`, `FROM`, etc. 

### The `select` transformation

For instance, the equivalent to the `map` typed transformation is `select`: 

In [15]:
val ds: Dataset[String] = peopleDs.map(_.name)
ds.collect
ds.show
ds.explain

collect at cmd14.sc:2

1 / 1

show at cmd14.sc:3

1 / 1

+-------+
|  value|
+-------+
|  YiHui|
| Javier|
|Gabriel|
| Noelia|
+-------+

== Physical Plan ==
*(1) SerializeFromObject [staticinvoke(class org.apache.spark.unsafe.types.UTF8String, StringType, fromString, input[0, java.lang.String, true], true, false) AS value#60]
+- *(1) MapElements ammonite.$sess.cmd14$Helper$$Lambda$5098/1786149424@1e8583f5, obj#59: java.lang.String
   +- *(1) DeserializeToObject newInstance(class ammonite.$sess.cmd12$Helper$Person), obj#58: ammonite.$sess.cmd12$Helper$Person
      +- *(1) FileScan json [age#29L,name#30] Batched: false, Format: JSON, Location: InMemoryFileIndex[file:/home/jovyan/work/data/people.json], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<age:bigint,name:string>


ds: Dataset[String] = [value: string]
res14_1: Array[String] = Array("YiHui", "Javier", "Gabriel", "Noelia")

In [16]:
val df: DataFrame = 
    spark.read.json("data/people.json").select($"name")
df.collect
df.show
df.schema

json at cmd15.sc:2

1 / 1

collect at cmd15.sc:3

1 / 1

show at cmd15.sc:4

1 / 1

+-------+
|   name|
+-------+
|  YiHui|
| Javier|
|Gabriel|
| Noelia|
+-------+



df: DataFrame = [name: string]
res15_1: Array[Row] = Array([YiHui], [Javier], [Gabriel], [Noelia])
res15_3: StructType = StructType(StructField("name", StringType, true, {}))

Note that we lost the column label (`name`) in the case of the dataset transformation. This is not happening with `select`. Moreover, we have more control over the resulting schema: 

In [17]:
peopleDs.map(p => (p.name, p.age + 1, p.name.substring(0,3)))
    .show

show at cmd16.sc:1

1 / 1

+-------+----+---+
|     _1|  _2| _3|
+-------+----+---+
|  YiHui|2001|YiH|
| Javier|2001|Jav|
|Gabriel|2004|Gab|
| Noelia|2005|Noe|
+-------+----+---+



In [18]:
people.select($"name", $"age" + 1 as "age", $"name".substr(0,3) as "prefix")
    .show

show at cmd17.sc:1

1 / 1

+-------+----+------+
|   name| age|prefix|
+-------+----+------+
|  YiHui|2001|   YiH|
| Javier|2001|   Jav|
|Gabriel|2004|   Gab|
| Noelia|2005|   Noe|
+-------+----+------+



The [org.apache.spark.sql.functions](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$) contains dozens of column operators.

Note that _untyped_, or more properly, _dynamically typed_, character means that the Scala compiler won't complain if we choose a non-existent column:

In [19]:
lazy val df: DataFrame = spark.read.json("data/people.json").select($"nam")

df: DataFrame = [last attempt failed]
org.apache.spark.sql.AnalysisException: cannot resolve '`nam`' given input columns: [age, name];;
'Project ['nam]
+- Relation[age#133L,name#134] json

  org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
  org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$$nestedInanonfun$checkAnalysis$1$2.applyOrElse(CheckAnalysis.scala:110)
  org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$$nestedInanonfun$checkAnalysis$1$2.applyOrElse(CheckAnalysis.scala:107)
  org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$transformUp$2(TreeNode.scala:278)
  org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:70)
  org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:278)
  org.apache.spark.sql.catalyst.plans.QueryPlan.$anonfun$transformExpressionsUp$1(QueryPlan.scala:93)
  org.apache.spark.sql.catalyst.plans.QueryPlan.$anonfun$mapExpressions$1(QueryPlan.scala:105)
  org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:70)
  org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:105)
  org.apache.spark.sql.catalyst.plans.QueryPlan.recursiveTransform$1(QueryPlan.scala:116)
  org.apache.spark.sql.catalyst.plans.QueryPlan.$anonfun$mapExpressions$3(QueryPlan.scala:121)
  scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
  scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
  scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
  scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
  scala.collection.TraversableLike.map(TraversableLike.scala:238)
  scala.collection.TraversableLike.map$(TraversableLike.scala:231)
  scala.collection.AbstractTraversable.map(Traversable.scala:108)
  org.apache.spark.sql.catalyst.plans.QueryPlan.recursiveTransform$1(QueryPlan.scala:121)
  org.apache.spark.sql.catalyst.plans.QueryPlan.$anonfun$mapExpressions$4(QueryPlan.scala:126)
  org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:187)
  org.apache.spark.sql.catalyst.plans.QueryPlan.mapExpressions(QueryPlan.scala:126)
  org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpressionsUp(QueryPlan.scala:93)
  org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$1(CheckAnalysis.scala:107)
  org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$1$adapted(CheckAnalysis.scala:85)
  org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:127)
  org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis(CheckAnalysis.scala:85)
  org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis$(CheckAnalysis.scala:82)
  org.apache.spark.sql.catalyst.analysis.Analyzer.checkAnalysis(Analyzer.scala:95)
  org.apache.spark.sql.catalyst.analysis.Analyzer.$anonfun$executeAndCheck$1(Analyzer.scala:108)
  org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:201)
  org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:105)
  org.apache.spark.sql.execution.QueryExecution.analyzed$lzycompute(QueryExecution.scala:57)
  org.apache.spark.sql.execution.QueryExecution.analyzed(QueryExecution.scala:55)
  org.apache.spark.sql.execution.QueryExecution.assertAnalyzed(QueryExecution.scala:47)
  org.apache.spark.sql.Dataset$.ofRows(Dataset.scala:78)
  org.apache.spark.sql.Dataset.org$apache$spark$sql$Dataset$$withPlan(Dataset.scala:3406)
  org.apache.spark.sql.Dataset.select(Dataset.scala:1335)
  ammonite.$sess.cmd18$Helper.$anonfun$df$value$1(cmd18.sc:1)
  almond.api.internal.Lazy.liftedTree1$1(Lazy.scala:10)
  almond.api.internal.Lazy.value$lzycompute(Lazy.scala:10)
  almond.api.internal.Lazy.value(Lazy.scala:8)
  ammonite.$sess.cmd19$Helper.<init>(cmd19.sc:1)
  ammonite.$sess.cmd19$.<init>(cmd19.sc:7)
  ammonite.$sess.cmd19$.<clinit>(cmd19.sc:-1)

The error will be shown at runtime: 

In [20]:
df

json at cmd18.sc:1

1 / 1

: 

On the contrary, the error in the dataset transformation manifests at compile-time:

In [20]:
peopleDs.map(_.nam)

cmd20.sc:1: value nam is not a member of cmd20.this.cmd12.Person
val res20 = peopleDs.map(_.nam)
                           ^Compilation Failed

: 

### The `filter` transformation

This is the equivalent to the typed `filter` transformation:

In [21]:
people.filter($"age" > 2001)
    .show

show at cmd20.sc:1

1 / 1

+----+-------+
| age|   name|
+----+-------+
|2003|Gabriel|
|2004| Noelia|
+----+-------+



If we pass a column function not denoting a boolean value, we won't even get a run-time exception:

In [22]:
def df: DataFrame = 
    people.filter($"name" > 2001)

defined function df

In [23]:
df.show

show at cmd22.sc:1

1 / 1

+---+----+
|age|name|
+---+----+
+---+----+



### The `groupBy` transformation

In [ ]:
val students: DataFrame = spark.read.json("data/students.json")

In [ ]:
students.groupBy($"degree").count.show

### `Join` transformations

We already discussed joins, but we didn't mention that the resulting type of a join is a dataframe, not a dataset: 

In [ ]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)

case class Student(name: String, degree: String)

In [ ]:
peopleDs.join(students.as[Student], "name")

# The problems of `Dataset`s

Datasets are nice because they are type safe, but, unfortunately, they are less efficient than data frames in several respects. This can be best shown by reading from parquet source files. 

Parquet is a _columnar_ format, which means that it stores physically data around columns, allowing us to read only data from a particular column without reading the entire row.

In [ ]:
people.write.mode("overwrite").parquet("data/people.parquet")

In [ ]:
spark.read.parquet("data/people.parquet").schema

### The `ReadSchema` optimization

Let's create a program that simply read the _name_ column of the people dataset:

In [ ]:
val ds: Dataset[String] = 
    spark.read.parquet("data/people.parquet").as[Person]
        .map(_.name)

which works as intended: 

In [ ]:
ds.show

We have a problem, however: 

In [ ]:
ds.explain

As we can see, the plan includes the directive `ReadSchema: struct<age:bigint,name:string>`, which generates a query to scan the full schema of the parquet file. But we just want to read the names! We can create an optimun program using dataframes:

In [ ]:
val df: DataFrame = 
    spark.read.parquet("data/people.parquet").select($"name")

which works similarly: 

In [ ]:
df.show

but more efficiently (note the the value of the `ReadSchema` directive):

In [ ]:
df.explain

We can empirically check that it actually works using the Spark UI. First, we create a parquet file with enough rows and several columns:

In [ ]:
spark.range(0, 1000000)
    .select($"id" as "_1", lit(1) as "_2")
    .write.mode("overwrite").parquet("data/test")

Now, we read the second column using both datasets and dataframes, and check the Spark UI for the _Input Size_ field.

In [ ]:
val test = spark.read.parquet("data/test")
test.as[Tuple2[Long, Int]].map(_._2).collect

Using dataframes the input size is much lower since we only read the second column:

In [ ]:
test.select($"_2").collect

### The `PushedFilter` optimization

Let's consider the following equivalent dataset and dataframe programs: 

In [ ]:
val ds: Dataset[(Long, Int)] = 
    test.as[(Long, Int)]
        .filter(_._1 >= 999995)

val df: DataFrame = 
    test
        .filter($"_1" >= 999995)

Functionally, they are equivalent, but their performance differ significantly:

In [ ]:
df.collect
ds.collect

The explanation of this difference lies in another optimization applied by the Spark SQL compiler: the so-called push-down filter optimization. In the previous `ReadSchema` optimization, we skipped certain columns of the dataset; now, we skip rows and read only the ones we are interested in (those that satisfy the predicate). We can check if the push-down filter optimization is actually applied by inspecting the query plan. 

In [ ]:
df.explain
ds.explain

### The `PartitionFilters` optimization

Let's create a test file with an additional column: 

In [ ]:
spark.range(0, 1000000)
    .select($"id" as "_1", lit(1) as "_2", round(rand() * 10) mod lit(10) as "_3")
    .write.mode("overwrite").parquet("data/test")

In [ ]:
val test: DataFrame = spark.read.parquet("data/test")

In [ ]:
test.show

Let's suppose that we want to read data with value `_3` equal to `9.0`:

In [ ]:
test.filter($"_3" === lit(9.0)).show

A pushed filter optimization is created, but it would be better if we could just read directly those rows with the exact value for the thrid column. We can achieve that as follows:

In [ ]:
test.write.mode("overwrite").partitionBy("_3").parquet("data/testP")

As we can see, the parquet file is splitted into ten partitions. Now, if we just want to process data with a particular key, Spark will generate an optimun query: 

In [ ]:
val testP: DataFrame = spark.read.parquet("data/testP")

In [ ]:
testP.filter($"_3" === lit(9.0)).show

We can inspet the Spark UI to check that we read less data in the last action.